<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/PyTorch_Blocks_and_Layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
import torch
from torch import nn
from torch.nn import functional as F

Some part of the code is based on d2l.ai book: http://d2l.ai/

In [170]:
### basing on the d2l.ai book 

In [171]:
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))

X=torch.rand(2,20)
net(X)

tensor([[ 0.0237,  0.0474,  0.0385,  0.1019,  0.1441,  0.0250, -0.0526,  0.0707,
         -0.0800, -0.1745],
        [ 0.0633, -0.0320,  0.0300, -0.0549, -0.0336, -0.0589, -0.0714,  0.0758,
         -0.0505, -0.0735]], grad_fn=<AddmmBackward>)

In [172]:
net.forward(X)

tensor([[ 0.0237,  0.0474,  0.0385,  0.1019,  0.1441,  0.0250, -0.0526,  0.0707,
         -0.0800, -0.1745],
        [ 0.0633, -0.0320,  0.0300, -0.0549, -0.0336, -0.0589, -0.0714,  0.0758,
         -0.0505, -0.0735]], grad_fn=<AddmmBackward>)

In [173]:
class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # Hidden layer
        self.out = nn.Linear(256, 10)  # Output layer

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


In [174]:
net = MLP()
net(X)

tensor([[-0.0248,  0.0502,  0.0443,  0.0466, -0.0601, -0.1229, -0.1250,  0.1480,
         -0.2301, -0.0225],
        [-0.0716,  0.2107, -0.0651,  0.1048,  0.0715, -0.0720, -0.1693,  0.1391,
         -0.2616, -0.1163]], grad_fn=<AddmmBackward>)

In [175]:
### Custom Sequential Block

In [176]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # Here, `block` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        for block in self._modules.values():
            X = block(X)
        return X



In [177]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0182, -0.1181,  0.2365, -0.1026, -0.0590, -0.1144,  0.0104,  0.0275,
          0.2318,  0.0266],
        [-0.1175,  0.0118,  0.2307,  0.0053,  0.0411, -0.2913, -0.0477, -0.0288,
          0.0823,  0.0643]], grad_fn=<AddmmBackward>)

MyParallel model: several models use the same input, executed in paralle and the results of the models are concatenated in one output.

In [178]:
class MyParallel(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # Here, `block` is an instance of a `Module` subclass. We save it
            # in the member variable `_modules` of the `Module` class, and its
            # type is OrderedDict
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict guarantees that members will be traversed in the order
        # they were added
        list_res=[]
        for block in self._modules.values():
          list_res.append(block(X))
        concat_fin=torch.cat(list_res, dim = -1)
        return concat_fin



In [179]:
list_models = [nn.Linear(20,20),nn.Linear(20,20),nn.Linear(20,20)]

In [180]:
net = MyParallel(nn.Linear(20,20),nn.Linear(20,20),nn.Linear(20,20))
#X = torch.tensor([1,10])
net(X)

tensor([[ 0.1293,  0.3620, -0.2750,  0.3550,  0.1753,  0.2276, -0.0453,  0.3909,
          0.3101,  0.1720, -0.2783, -0.1499,  0.2624, -0.2230, -0.0739, -0.5524,
          0.0153,  0.4702,  0.0078, -0.2402, -0.0538, -0.1591, -0.2021,  0.3624,
          0.2835, -0.3667, -0.1555,  0.0529,  0.4494, -0.3535, -0.2167,  0.1674,
          0.3753,  0.3594, -0.1101, -0.0159,  0.1857,  0.4056,  0.3484,  0.2011,
         -0.8248,  0.4090, -0.0222, -0.0134, -0.2785, -0.0625, -0.4550,  0.4799,
          0.0025,  0.3386, -0.4627,  0.0014, -0.1049, -0.0839,  0.0616, -0.0263,
         -0.0181, -0.2403,  0.7301, -0.1425],
        [ 0.1376,  0.3970, -0.0365,  0.6587, -0.0906,  0.2326, -0.3146, -0.0177,
          0.2065,  0.1578,  0.0982, -0.0815,  0.1859, -0.1964, -0.4040, -0.2211,
         -0.1409,  0.2076, -0.1940, -0.4306, -0.3078, -0.0205, -0.2873, -0.1400,
          0.1774, -0.0252,  0.1238,  0.1058,  0.4679,  0.2104, -0.1621,  0.1129,
          0.3291,  0.7106, -0.0405,  0.0251,  0.3324,  0.3404, 

In [181]:
net = MyParallel(*list_models)
net(X)

tensor([[-0.0414,  0.1768, -0.2009,  0.2508,  0.3385,  0.7083,  0.2919,  0.0889,
          0.2914, -0.6532, -0.5163, -0.1289, -1.0221,  0.3668, -0.1724,  0.3483,
          0.0373,  0.3762,  0.0551,  0.2034,  0.2387,  0.2787,  0.5693, -0.2542,
         -0.2130, -0.7880,  0.0790,  0.0994, -0.1545,  0.5005, -0.2365, -0.0567,
         -0.6380,  0.5348,  0.3780,  0.0419,  0.5773,  0.0337,  0.0403, -0.4502,
         -0.3470,  0.1956, -0.1440, -0.0644, -0.4514, -0.3856, -0.2550,  0.1978,
          0.3300,  0.1284, -0.5059,  0.3016,  0.1725, -0.3960, -0.5497, -0.8357,
         -0.5133,  0.2965, -0.0821, -0.0141],
        [ 0.1638,  0.1156,  0.0209,  0.4286,  0.5214,  0.5983,  0.5789,  0.1002,
          0.6894, -0.7016, -0.6722, -0.0029, -1.2186,  0.1583, -0.1656,  0.7432,
          0.2936,  0.4025, -0.0087,  0.1440,  0.0942,  0.2580,  0.8077, -0.3458,
         -0.2411, -0.2847,  0.2503, -0.0695,  0.1638,  0.3611, -0.2980, -0.0682,
         -0.1518,  0.4896,  0.4077,  0.1814,  0.4822,  0.2675, 

In [182]:
##### Nesting Modules
class NestedBlocks(nn.Module):
  def __init__(self,*args):
    super().__init__()
    self.net = nn.Sequential(nn.Linear(20,20),nn.Linear(20,20))
    self.linear = nn.Linear(20,20)

  def forward(self, X):
    return self.linear(self.net(X))



In [183]:
tensor = torch.ones([2,20])
net = NestedBlocks()
net(X)

tensor([[ 0.0937, -0.1466,  0.3278,  0.1944, -0.0631, -0.2502,  0.0113, -0.1054,
          0.1111,  0.0151, -0.0034,  0.0767,  0.2366, -0.1687, -0.1052,  0.1522,
          0.4037, -0.1529, -0.0058, -0.2199],
        [-0.0887, -0.3235,  0.3774,  0.3524,  0.0164, -0.2565, -0.0706, -0.1376,
          0.1584, -0.0275,  0.0340, -0.0673,  0.2173, -0.1887, -0.0955,  0.0606,
          0.2554, -0.1287, -0.0249, -0.1184]], grad_fn=<AddmmBackward>)

Parameters access

In [184]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X=torch.rand((2,4))
net(X)

tensor([[-0.4657],
        [-0.3875]], grad_fn=<AddmmBackward>)

In [185]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0403, -0.2438, -0.2953,  0.0009,  0.0137, -0.0172, -0.1540, -0.2843]])), ('bias', tensor([-0.3019]))])


In [186]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3019], requires_grad=True)
tensor([-0.3019])


In [187]:
net[2].weight.grad == True

False

In [188]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [189]:
net.state_dict()['0.weight'].data

tensor([[-0.0208,  0.3736,  0.1639, -0.2877],
        [-0.2837,  0.1746,  0.3658, -0.3744],
        [-0.4959, -0.0830, -0.4249, -0.1329],
        [ 0.0738, -0.3496,  0.0964, -0.3045],
        [-0.1577,  0.4065,  0.2789,  0.3619],
        [ 0.2017, -0.2920, -0.3666, -0.4722],
        [-0.2690, -0.0681, -0.3297,  0.4311],
        [ 0.2925, -0.3944, -0.2464,  0.2031]])

In [190]:
def block1():
  return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                        nn.Linear(8, 4), nn.ReLU())

def block2():
  net = nn.Sequential()
  for i in range(4):
    net.add_module(f'block {i}', block1())
  return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[-0.1365],
        [-0.1365]], grad_fn=<AddmmBackward>)

In [191]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [192]:
"""
net_prl = MyParallel(nn.Linear(4,4),nn.Linear(4,4),nn.Linear(4,4))
X=torch.rand(1,4)
net_prl(X)
print(net_prl)
"""

'\nnet_prl = MyParallel(nn.Linear(4,4),nn.Linear(4,4),nn.Linear(4,4))\nX=torch.rand(1,4)\nnet_prl(X)\nprint(net_prl)\n'

In [193]:
rgnet[0][1][0].bias.data

tensor([ 0.2354,  0.1424, -0.2331,  0.0673, -0.3973, -0.3581,  0.3154,  0.1150])

Parameter Initialization

In [194]:
def init_normal(m):
  if type(m)==nn.Linear:
    nn.init.normal_(m.weight, mean=0, std=0.01)
    nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data , net[0].bias.data

(tensor([[-0.0076, -0.0019, -0.0037, -0.0075],
         [-0.0152,  0.0067, -0.0066, -0.0058],
         [-0.0119, -0.0104,  0.0149,  0.0007],
         [-0.0150, -0.0013,  0.0046, -0.0057],
         [-0.0088, -0.0161,  0.0025,  0.0009],
         [-0.0004,  0.0104,  0.0252,  0.0083],
         [ 0.0017,  0.0016,  0.0121,  0.0024],
         [ 0.0004,  0.0177, -0.0004,  0.0069]]),
 tensor([0., 0., 0., 0., 0., 0., 0., 0.]))

In [195]:
def xavier(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        torch.nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data)
print(net[2].weight.data)

tensor([[-0.4120, -0.3794, -0.0735, -0.6707],
        [-0.0843, -0.0587, -0.6795, -0.5272],
        [-0.1224,  0.0148, -0.1513, -0.3396],
        [-0.0338, -0.3446,  0.0796, -0.5932],
        [ 0.1550,  0.4798,  0.1734,  0.2339],
        [-0.1635,  0.1171, -0.0360,  0.4638],
        [-0.0273,  0.0631,  0.1151, -0.2536],
        [-0.0516, -0.3743,  0.5188, -0.1587]])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [196]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Parameter containing:
tensor([[-0.0000, -0.0000,  0.0000,  0.0000],
        [-9.2048, -0.0000, -9.7863,  0.0000],
        [-6.2107, -0.0000,  0.0000, -9.1236],
        [-5.0346, -0.0000, -0.0000,  6.1767],
        [-5.8910, -0.0000, -7.3705,  0.0000],
        [ 0.0000,  0.0000, -7.6496,  5.9479],
        [-9.5257, -0.0000,  0.0000,  8.3728],
        [-0.0000, -9.1081, -0.0000,  6.8390]], requires_grad=True)

In [197]:
### we can set parameters
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.,  1.,  1.,  1.])

In [198]:
### shared layers 
### give a name
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
### has same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


In [199]:
### actually same objects
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


In [200]:
### without parameters
class CenteredLayer(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,X):
    return (X-X.mean())


In [201]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5,6]))

tensor([-2.5000, -1.5000, -0.5000,  0.5000,  1.5000,  2.5000])

In [202]:
net = nn.Sequential(nn.Linear(5,5), CenteredLayer())
Y = net(torch.rand(2,5))
Y.mean()

tensor(1.4901e-09, grad_fn=<MeanBackward0>)

In [203]:
### Linear from scratch
class MyLinear(nn.Module):
  def __init__(self, in_units, out_units):
    super().__init__()
    self.weight=nn.Parameter(torch.randn(in_units, out_units))
    self.bias = nn.Parameter(torch.randn(out_units))

  def forward(self,X):
    linear = torch.matmul(X, self.weight.data)+ self.bias.data
    return F.relu(linear)


In [204]:
dense = MyLinear(5,3)
dense(torch.randn(2,5))

tensor([[2.2841, 0.0000, 0.0000],
        [0.0000, 3.2201, 1.6283]])

In [205]:
net = nn.Sequential(MyLinear(10,5), MyLinear(5,3))
net(torch.randn(2,10))

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [206]:
#### saving results into  file 
x = torch.arange(5)
torch.save(x, 'x_file')

In [207]:
x2 = torch.load('x_file')
x2

tensor([0, 1, 2, 3, 4])

In [208]:
y = torch.zeros(4)
torch.save([x,y], 'x_file')

In [209]:
l,m = torch.load('x_file')
(l,m)

(tensor([0, 1, 2, 3, 4]), tensor([0., 0., 0., 0.]))

In [210]:
### read write from dict
myDict = {'x':x, 'y':y}
torch.save(myDict, 'mydict')
mydict2 = torch.load('mydict')
mydict2


{'x': tensor([0, 1, 2, 3, 4]), 'y': tensor([0., 0., 0., 0.])}

In [211]:
### save model parameters
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [212]:
torch.save(net.state_dict(), 'mlp.params')

In [213]:
### clone model and load params to the clone
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))

<All keys matched successfully>

In [214]:
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [215]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [216]:
!nvidia-smi

Wed Dec  2 15:39:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    58W / 149W |    395MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [217]:
torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x7fb50cde0940>)

In [218]:
torch.cuda.device_count()

1

In [219]:
def try_gpu(i=0):
  if torch.cuda.device_count() >= i+1:
    return torch.device(f'cuda:{i}')
  return torch.device('cpu')

def try_all_gpus():
  devices = [torch.device(f'cuda:{i}') 
            for i  in range (torch.cuda.device_count())]
  return devices if devices else [torch.device('cpu')]

  try_gpu(), try_gpu(2), try_all_gpus()

In [220]:
x= torch.tensor([1,2,3])
x.device

device(type='cpu')

In [221]:
X = torch.ones(2,3, device = try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [222]:
Y = torch.rand(2, 3, device=try_gpu(0))
Y

tensor([[0.6576, 0.0485, 0.3303],
        [0.2826, 0.6320, 0.7394]], device='cuda:0')

In [223]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [224]:
Y + Z

tensor([[1.6576, 1.0485, 1.3303],
        [1.2826, 1.6320, 1.7394]], device='cuda:0')